In [1]:
!pip install spacy


In [2]:
!pip install textblob


In [1]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import spacy
from textblob import TextBlob
from collections import Counter
import re

# Loading the model
nlp = spacy.load("en_core_web_sm")

# Our review from our recommendation
reviews = [
    "I love Italian food, especially pasta and pizza. I dislike too much cheese on my pizza. I would enjoy this as a dinner.",
    "Vegetarian food is amazing, but I need gluten-free options. I enjoyed the quinoa salad for lunch.",
    "I prefer quick meals like stir-fried vegetables. I don’t like spicy food, though, I enjoy a light snack."
]

# Doing data preprocessing
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()

# Classifying sentiments based on polarity(positive,negative and neutral)
def classify_sentiment(polarity):
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'


def extract_meal_types(review):
    meal_keywords = ['breakfast', 'lunch', 'dinner', 'snack', 'meal', 'dessert', 'brunch']
    meal_types = [word for word in review.split() if word.lower() in meal_keywords]
    return meal_types

# Function to extract preferences from reviews
def extract_preferences_with_sentiment(review):
    cleaned_review = clean_text(review)
    doc = nlp(cleaned_review)

    preferences = {
        'cuisines': [],
        'ingredients': [],
        'meal_types': [],
        'sentiments': [],
        'others': []
    }

    cuisines_keywords = ['italian', 'chinese', 'indian', 'mexican', 'french', 'japanese', 'thai']
    ingredients_keywords = ['pasta', 'pizza', 'tomato', 'cheese', 'quinoa', 'salad', 'spinach', 'carrot', 'chicken', 'tofu']

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            preferences['cuisines'].append(ent.text)
        elif ent.label_ == 'PRODUCT':  #maybe ingredient or dish
            preferences['ingredients'].append(ent.text)
        else:
            preferences['others'].append(ent.text)

    for word in cleaned_review.split():
        if word in cuisines_keywords:
            preferences['cuisines'].append(word)
        elif word in ingredients_keywords:
            preferences['ingredients'].append(word)

    preferences['meal_types'] = extract_meal_types(review)

    sentences = cleaned_review.split(".")
    for sentence in sentences:
        sentiment = TextBlob(sentence).sentiment
        sentiment_label = classify_sentiment(sentiment.polarity)
        preferences['sentiments'].append(sentiment_label)  # Positive, Negative, or Neutral

    return preferences

# Running the code
for review in reviews:
    preferences = extract_preferences_with_sentiment(review)
    print(f"Review: {review}")
    print("Extracted Preferences:")
    print(preferences)
    print("="*40)


Review: I love Italian food, especially pasta and pizza. I dislike too much cheese on my pizza. I would enjoy this as a dinner.
Extracted Preferences:
{'cuisines': ['italian'], 'ingredients': ['pasta', 'pizza', 'cheese', 'pizza'], 'meal_types': [], 'sentiments': ['positive'], 'others': ['italian']}
Review: Vegetarian food is amazing, but I need gluten-free options. I enjoyed the quinoa salad for lunch.
Extracted Preferences:
{'cuisines': [], 'ingredients': ['quinoa', 'salad'], 'meal_types': [], 'sentiments': ['positive'], 'others': []}
Review: I prefer quick meals like stir-fried vegetables. I don’t like spicy food, though, I enjoy a light snack.
Extracted Preferences:
{'cuisines': [], 'ingredients': [], 'meal_types': [], 'sentiments': ['positive'], 'others': []}


In [9]:
!pip install jupyter-dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [11]:
import dash
from dash import dcc, html, dash_table
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash  # Import JupyterDash instead of Dash



In [13]:
import dash
from dash import dcc, html, dash_table
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash

# Extended recommendations with different cuisines
recommendations = pd.DataFrame({
    "title": [
        "Spaghetti Carbonara", "Margarita Pizza", "Pesto Pasta",
        "Tacos", "Guacamole", "Chili Con Carne",
        "Sweet and Sour Chicken", "Kung Pao Chicken", "Spring Rolls",
        "Butter Chicken", "Aloo Gobi", "Chicken Tikka Masala"
    ],
    "ingredients": [
        "spaghetti, eggs, pancetta", "pizza dough, mozzarella, basil", "pasta, basil, pine nuts",
        "taco shells, ground beef, lettuce, tomato, cheese", "avocados, lime, cilantro, salt", "ground beef, beans, tomatoes, chili powder",
        "chicken, vinegar, sugar, soy sauce", "chicken, peanuts, soy sauce, bell peppers", "spring roll wrappers, vegetables, soy sauce",
        "chicken, butter, cream, garam masala", "potatoes, cauliflower, turmeric, cumin", "chicken, yogurt, garam masala, tomatoes"
    ],
    "directions": [
        "Cook spaghetti, mix with eggs and pancetta", "Bake pizza dough with toppings", "Blend basil with pine nuts, add pasta",
        "Cook beef and assemble tacos with toppings", "Mash avocados with lime and cilantro", "Cook beef, add beans and spices",
        "Cook chicken, mix with sweet and sour sauce", "Stir-fry chicken and bell peppers, add peanuts", "Fry spring rolls with vegetables",
        "Cook chicken in butter and cream, add garam masala", "Cook potatoes and cauliflower with spices", "Cook chicken in curry sauce"
    ],
    "cuisine": [
        "Italian", "Italian", "Italian",
        "Mexican", "Mexican", "Mexican",
        "Chinese", "Chinese", "Chinese",
        "Indian", "Indian", "Indian"
    ]
})

# Ingredient bar chart
ingredient_data = recommendations['ingredients'].str.split(', ').explode().value_counts()
ingredient_chart = px.bar(
    ingredient_data,
    x=ingredient_data.index,
    y=ingredient_data.values,
    labels={'x': 'Ingredients', 'y': 'Frequency'},
    title='Ingredient Usage Across Recommendations'
)

# Cuisine type pie chart
cuisine_chart = px.pie(
    recommendations,
    names='cuisine',
    title='Cuisine Distribution in Recommendations'
)

# Dash app
app = dash.Dash(__name__)  # Use Dash without JupyterDash

app.layout = html.Div([
    html.H1("Recipe Recommendation Dashboard"),

    html.H2("Recommendations Table"),
    dash_table.DataTable(
        data=recommendations.to_dict('records'),
        columns=[{"name": col, "id": col} for col in recommendations.columns],
        style_table={'overflowX': 'auto'}
    ),

    html.H2("Ingredient Frequency"),
    dcc.Graph(figure=ingredient_chart),

    html.H2("Cuisine Distribution"),
    dcc.Graph(figure=cuisine_chart),
])

# Run the app in Colab
app.run_server(debug=True, use_reloader=False)


<IPython.core.display.Javascript object>